In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.sparse import csr_matrix

In [3]:
# Specify the path to your Parquet file
file_path = 'data/2023/yellow_tripdata_2023-01.parquet'

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)

In [4]:
# Get the number of columns
num_columns = df.shape[1]

print(f'Number of columns: {num_columns}')

Number of columns: 19


In [6]:
# Convert columns to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Calculate the duration in minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [7]:
df['duration']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [8]:
# Calculate the standard deviation of the duration column
duration_std = df['duration'].std()

# Print the standard deviation
print(f'Standard Deviation of Duration: {duration_std} minutes')

Standard Deviation of Duration: 42.59435124195458 minutes


In [9]:
# Filter the DataFrame to keep only records where the duration is between 1 and 60 minutes
filtered_df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records remaining
fraction_remaining = len(filtered_df) / len(df)

# Print the fraction of records remaining
print(f'Fraction of records remaining after removing outliers: {fraction_remaining}')

Fraction of records remaining after removing outliers: 0.9812202822125979


In [10]:
filtered_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [11]:
# Convert 'PULocationID' and 'DOLocationID' to strings
filtered_df['PULocationID'] = filtered_df['PULocationID'].astype(str)
filtered_df['DOLocationID'] = filtered_df['DOLocationID'].astype(str)

# Select only the 'PULocationID' and 'DOLocationID' columns for the model
location_df = filtered_df[['PULocationID', 'DOLocationID']]

In [12]:
location_df.head()

,PULocationID,DOLocationID
0,161,141
1,43,237
2,48,238
3,138,7
4,107,79


In [13]:
# Turn the DataFrame into a list of dictionaries
location_dicts = location_df.to_dict(orient='records')

In [14]:
# Fit a dictionary vectorizer
dv = DictVectorizer(sparse=False)
X = dv.fit_transform(location_dicts)

In [15]:
X.shape

(3009173, 515)

In [16]:
# Target variable
y = filtered_df['duration'].values

In [17]:
# Convert to sparse matrix
X = csr_matrix(X)

In [18]:
# Train a plain linear regression model
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [19]:
# Predict on the training data
y_pred = model.predict(X)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))

# Print the RMSE
print(f'RMSE: {rmse}')

RMSE: 7.649261936284003


## Validation

In [20]:
# Validation data
file_path = 'data/2023/yellow_tripdata_2023-02.parquet'
df_val = pd.read_parquet(file_path)

In [21]:
# Convert columns to datetime
df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])

# Calculate the duration in minutes
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter the DataFrame to keep only records where the duration is between 1 and 60 minutes
filtered_df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]

# Convert 'PULocationID' and 'DOLocationID' to strings
filtered_df_val['PULocationID'] = filtered_df_val['PULocationID'].astype(str)
filtered_df_val['DOLocationID'] = filtered_df_val['DOLocationID'].astype(str)

# Select only the 'PULocationID' and 'DOLocationID' columns for the model
location_df_val = filtered_df_val[['PULocationID', 'DOLocationID']]

# Turn the DataFrame into a list of dictionaries
location_dicts_val = location_df_val.to_dict(orient='records')

In [22]:
# Transform validation data with training transformer
X_val = dv.transform(location_dicts_val)

In [23]:
# Target variable
y_val = filtered_df_val['duration'].values

In [24]:
y_val_pred = model.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))

print(f'Validation RMSE: {rmse_val}')

Validation RMSE: 7.811818654341152
